# Pré Processamento da Base de Dados

#### Caminho para salvar as bases de dados

In [116]:
import os
out = os.path.basename('C:/Users/guga_/Desafio Geofusion/Dados')
print(out)


Dados


#### Funções

In [117]:
def Salva_DB(Caminho,Filename,Arquivo):
    open_file = open(f'{Caminho}/{Filename}', "wb")
    pickle.dump(Arquivo, open_file)
    open_file.close()

def Abre_DB(Caminho,Filename):
    with open(f'{Caminho}/{Filename}','rb') as f:
        DB = pickle.load(f)
    return DB

#### Leitura da Base de dado

In [118]:
import pandas as pd

DB = pd.read_excel(f'{out}/DadosDesafioCientista.xlsx')

#### Separar Base de dados entre RJ e SP

In [119]:
DB_RJ = DB[DB['estado']=='RJ']
DB_SP = DB[DB['estado']=='SP']

#### Salvar as Bases de dados

In [120]:
import pickle
Salva_DB(out,'Base_SP.pkl',DB_SP)
Salva_DB(out,'Base_RJ.pkl',DB_RJ)

### Pré Processamento da Base RJ

#### A Base do RJ será utilizada para duas ocasiões:
- Realizar uma previsão do faturamento das lojas de SP;
- Classificar o potencial de uma loja entre Alto, Médio e Baixo;

#### Para as duas ocasiões será necessário:
- Preencher os valores NaN (6) com a média da renda, não alterando a distribuição;
- Retirar as colunas codigo, nome, cidade e estado;
- Separar as colunas faturamento e potencial;
- Padronizar e normalizar os dados existentes para que os modelos possam se adequar melhor a base de dados.
    - Será utilizado o StandardScaler da biblioteca Scikit Learn;
- Separar a base de dados entre Treino e Teste para a avaliar o desempenho dos modelos;
    - Será utilizado o train_test_split da biblioteca Scikit Learn;
- Salvar as bases de Treino e Teste para utilizar posteriormente;
    

#### Leitura da Base do RJ

In [121]:
DB = Abre_DB(out,'Base_RJ.pkl')

#### Preenchendo NaN

In [122]:
DB['rendaMedia']=DB['rendaMedia'].fillna(DB['rendaMedia'].mean())

#### Retirar as colunas indesejadas

In [123]:
DB_dados_faturamento = DB.iloc[:,4:-2]
DB_dados_potencial = DB.iloc[:,4:-1]

#### Encoding da variável de potencial

In [124]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

potencial_encoded = labelencoder.fit_transform(DB.iloc[:,-1])


mymap = {'0':'Alto', '1':'Baixo', '2':'Médio'}

#### Padronização da Base de dados

Utilizando a função:

pd.Dataframe(DB).describe()

é possível verificar a padronização dos dados, onde:
- média = 0 
- desvio = 1

In [125]:
from sklearn.preprocessing import StandardScaler
import numpy as np
scaler_faturamento = StandardScaler()
scaler_potencial = StandardScaler()

DB_Scaled_faturamento = scaler_faturamento.fit_transform(DB_dados_faturamento)

print('Média:',np.round(DB_Scaled_faturamento.mean(axis=0)))
print('Desvio:',np.round(DB_Scaled_faturamento.std(axis=0)))

DB_Scaled_potencial = scaler_potencial.fit_transform(DB_dados_potencial)

print('Média:',np.round(DB_Scaled_potencial.mean(axis=0)))
print('Desvio:',np.round(DB_Scaled_potencial.std(axis=0)))


Média: [ 0. -0.  0.  0.  0. -0.  0.  0. -0. -0.  0.  0.  0.  0. -0. -0.  0.  0.]
Desvio: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Média: [ 0. -0.  0.  0.  0. -0.  0.  0. -0. -0.  0.  0.  0.  0. -0. -0.  0.  0.
  0.]
Desvio: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


#### Separar as colunas de faturamento e potencial

In [126]:
faturamento = DB.iloc[:,-2]
potencial = DB.iloc[:,-1]


#### Separar a base de dados em Treino e Teste

In [127]:
from sklearn.model_selection import train_test_split

X_train_potencial, X_test_potencial, y_train_potencial, y_test_potencial = train_test_split(
    DB_Scaled_potencial, potencial_encoded, test_size=0.15, random_state=42)
X_train_faturamento, X_test_faturamento, y_train_faturamento, y_test_faturamento = train_test_split(
    DB_Scaled_faturamento, faturamento, test_size=0.15, random_state=42)    


#### Salvar Bases de Treino e Teste

In [128]:
Salva_DB(f'{out}/Treino',f'X_train_potencial.pkl',X_train_potencial)
Salva_DB(f'{out}/Treino',f'y_train_potencial.pkl',y_train_potencial)
Salva_DB(f'{out}/Treino',f'X_train_faturamento.pkl',X_train_faturamento)
Salva_DB(f'{out}/Treino',f'y_train_faturamento.pkl',y_train_faturamento)
Salva_DB(f'{out}/Teste',f'X_test_potencial.pkl',X_test_potencial)
Salva_DB(f'{out}/Teste',f'y_test_potencial.pkl',y_test_potencial)
Salva_DB(f'{out}/Teste',f'X_test_faturamento.pkl',X_test_faturamento)
Salva_DB(f'{out}/Teste',f'y_test_faturamento.pkl',y_test_faturamento)

### Pré Processamento Base SP

Visualizando a base de forma geral, foi possível identificar que possuem 3 registros sem nenhuma entrada de informação ([232,361,376]).

O que é necessário fazer:
- Separar as colunas faturamento e potencial;
- Juntar a coluna de faturamento obtida pela previsão;
- Padronizar e normalizar os dados existentes para que os modelos possam se adequar melhor a base de dados.
    - Será utilizado o StandardScaler da biblioteca Scikit Learn;
- Salvar a base de dados para utilizar posteriormente;

#### Leitura da Base

In [129]:
DB_SP = Abre_DB(out,'Base_SP.pkl')

In [130]:
DB_SP=DB_SP.replace('-',0)

#### Separar as colunas de faturamento e potencial

In [131]:
DB_dados = DB_SP.iloc[:,4:-2]



#### Padronização antes do faturamento para base de faturamento

In [132]:
from sklearn.preprocessing import StandardScaler
import numpy as np
scaler = StandardScaler()

DB_Scaled = scaler.fit_transform(DB_dados)

print('Média:',np.round(DB_Scaled.mean(axis=0)))
print('Desvio:',np.round(DB_Scaled.std(axis=0)))

Média: [ 0.  0. -0.  0.  0. -0.  0. -0. -0. -0.  0.  0.  0.  0. -0.  0. -0.  0.]
Desvio: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


#### Salvar Base antes do faturamento

In [133]:
Salva_DB(f'{out}/Teste',f'Base_SP.pkl',DB_Scaled)

#### Juntar a coluna de faturamento gerada pelo modelo de regressão

In [134]:
DB_dados['faturamento'] = np.array(Abre_DB(out,'Faturamento_SP.pkl'))

In [135]:
DB_dados

,população,popAte9,popDe10a14,popDe15a19,popDe20a24,popDe25a34,popDe35a49,popDe50a59,popMaisDe60,domiciliosA1,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento
160,94034.0,12668.0,6853.0,9836.0,7487.0,14535.0,21549.0,10598.0,10508.0,0.0,253.0,2197.0,4368.0,6681.0,7011.0,2247.0,5670.0,1501.0,2.067560e+05
161,32791.0,2297.0,1017.0,2096.0,2197.0,5341.0,7281.0,4917.0,7645.0,1413.0,1734.0,3704.0,2351.0,1946.0,827.0,291.0,1617.0,5920.0,1.624085e+06
162,104193.0,15070.0,7343.0,10631.0,8657.0,17749.0,23364.0,11567.0,9812.0,0.0,0.0,1423.0,4875.0,8595.0,10082.0,3111.0,5776.0,1284.0,6.449765e+04
163,12721.0,953.0,343.0,627.0,819.0,2142.0,2833.0,1790.0,3214.0,624.0,667.0,1558.0,1032.0,915.0,361.0,84.0,404.0,6278.0,1.035049e+06
164,48417.0,5078.0,2396.0,4018.0,3571.0,7388.0,10751.0,6648.0,8567.0,0.0,303.0,1794.0,2986.0,4489.0,2836.0,1104.0,2553.0,1905.0,9.039367e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,34061.0,2908.0,1462.0,2253.0,2076.0,4579.0,7557.0,5227.0,7999.0,942.0,1089.0,2866.0,2219.0,2216.0,1227.0,306.0,1098.0,5285.0,1.463185e+06
452,35403.0,4127.0,1890.0,2678.0,2433.0,5855.0,9107.0,4608.0,4705.0,1726.0,1522.0,2458.0,1186.0,1166.0,918.0,209.0,3840.0,7418.0,1.857500e+06
453,122359.0,18304.0,9304.0,13258.0,9965.0,19248.0,26592.0,12579.0,13109.0,0.0,0.0,1758.0,4517.0,9450.0,11473.0,3218.0,7540.0,1252.0,6.226067e+04
454,125864.0,14670.0,7305.0,11225.0,9338.0,18841.0,28500.0,16700.0,19285.0,8.0,872.0,5093.0,8063.0,10012.0,8082.0,2856.0,6853.0,1936.0,1.063012e+06


#### Padronização da Base de dados

In [136]:
from sklearn.preprocessing import StandardScaler
import numpy as np
scaler = StandardScaler()

DB_Scaled = scaler.fit_transform(DB_dados)

print('Média:',np.round(DB_Scaled.mean(axis=0)))
print('Desvio:',np.round(DB_Scaled.std(axis=0)))


Média: [ 0.  0. -0.  0.  0. -0.  0. -0. -0. -0.  0.  0.  0.  0. -0.  0. -0.  0.
  0.]
Desvio: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


#### Salvar base padronizada para rodar o modelo de Regressão

In [137]:
DB_Scaled.shape

(296, 19)

In [138]:
Salva_DB(f'{out}',f'Base_SP_faturamento.pkl',DB_Scaled)

# Processamento Final

In [147]:
Base_Final = Abre_DB(out,'Base_SP.pkl')
mymap = {0:'Alto', 1:'Baixo', 2:'Médio'}
Base_Final['potencial']=((Abre_DB(out,'potencial.pkl')))
Base_Final['faturamento'] = np.array(Abre_DB(out,'Faturamento_SP.pkl'))


,codigo,nome,cidade,estado,população,popAte9,popDe10a14,popDe15a19,popDe20a24,popDe25a34,...,domiciliosA2,domiciliosB1,domiciliosB2,domiciliosC1,domiciliosC2,domiciliosD,domiciliosE,rendaMedia,faturamento,potencial
160,355030251.0,A. E. Carvalho,São Paulo,SP,94034.0,12668.0,6853.0,9836.0,7487.0,14535.0,...,253.0,2197.0,4368.0,6681.0,7011.0,2247.0,5670.0,1501.0,2.067560e+05,Médio
161,35503020.0,Aclimação,São Paulo,SP,32791.0,2297.0,1017.0,2096.0,2197.0,5341.0,...,1734.0,3704.0,2351.0,1946.0,827.0,291.0,1617.0,5920.0,1.624085e+06,Alto
162,355030285.0,Adventista,São Paulo,SP,104193.0,15070.0,7343.0,10631.0,8657.0,17749.0,...,0.0,1423.0,4875.0,8595.0,10082.0,3111.0,5776.0,1284.0,6.449765e+04,Médio
163,35503088.0,Água Branca,São Paulo,SP,12721.0,953.0,343.0,627.0,819.0,2142.0,...,667.0,1558.0,1032.0,915.0,361.0,84.0,404.0,6278.0,1.035049e+06,Médio
164,35503066.0,Água Funda,São Paulo,SP,48417.0,5078.0,2396.0,4018.0,3571.0,7388.0,...,303.0,1794.0,2986.0,4489.0,2836.0,1104.0,2553.0,1905.0,9.039367e+05,Médio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,355030213.0,Vila Sônia,São Paulo,SP,34061.0,2908.0,1462.0,2253.0,2076.0,4579.0,...,1089.0,2866.0,2219.0,2216.0,1227.0,306.0,1098.0,5285.0,1.463185e+06,Alto
452,355030207.0,Vila Suzana,São Paulo,SP,35403.0,4127.0,1890.0,2678.0,2433.0,5855.0,...,1522.0,2458.0,1186.0,1166.0,918.0,209.0,3840.0,7418.0,1.857500e+06,Alto
453,355030162.0,Vila Terezinha,São Paulo,SP,122359.0,18304.0,9304.0,13258.0,9965.0,19248.0,...,0.0,1758.0,4517.0,9450.0,11473.0,3218.0,7540.0,1252.0,6.226067e+04,Médio
454,355030157.0,Vila Zatt,São Paulo,SP,125864.0,14670.0,7305.0,11225.0,9338.0,18841.0,...,872.0,5093.0,8063.0,10012.0,8082.0,2856.0,6853.0,1936.0,1.063012e+06,Alto


In [148]:
Salva_DB(out,'BASE_FINAL_SP.pkl',Base_Final)